In [106]:
import pandas as pd
import numpy as np


In [107]:
# Load merged dataset
merged = pd.read_csv("/Users/ayush/Library/Mobile Documents/com~apple~CloudDocs/Developer/UIDAI/02_Cleaned_Data/merged.csv")

# Load ML model outputs
risk = pd.read_csv("/Users/ayush/Library/Mobile Documents/com~apple~CloudDocs/Developer/UIDAI/Outputs/risk_predictions.csv")


In [108]:
merged = merged.reset_index(drop=True)
risk = risk.reset_index(drop=True)

merged['risk_score'] = risk['risk_score']


In [ ]:
merged[['risk_score']].head()

,risk_score
0,0.007412
1,0.001601
2,0.025498
3,0.000987
4,0.858008


# Age Verification Model

In [ ]:
def age_verification_model(p_risk, is_adult, tau=0.85):
    
    if p_risk >= tau:
        return "DEFER_VERIFICATION"
    elif is_adult:
        return "AGE_VERIFIED_18_PLUS"
    else:
        return "AGE_NOT_VERIFIED"


In [111]:
merged['age_verification_decision'] = merged.apply(
    lambda row: age_verification_model(
        row['risk_score'],
        row['age_18_greater'] > 0
    ),
    axis=1
)


In [112]:
merged['age_verification_decision'].value_counts()


age_verification_decision
AGE_VERIFIED_18_PLUS    3534
AGE_NOT_VERIFIED         838
DEFER_VERIFICATION        26
Name: count, dtype: int64

# Disaster Alert Risk Fusion Model

In [120]:
merged['baseline_updates'] = (
    merged
    .groupby(['state','district'])['total_updates']
    .transform('mean')
)

merged['update_spike_ratio'] = (
    merged['total_updates'] /
    (merged['baseline_updates'] + 1)
)


In [ ]:
def disaster_alert_model(update_spike_ratio, risk_score):
    
    if update_spike_ratio >= 1.8 and risk_score >= 0.7:
        return "HIGH_ALERT"
    elif update_spike_ratio >= 1.5:
        return "WATCH"
    else:
        return "NORMAL"


In [122]:
merged['disaster_alert'] = merged.apply(
    lambda row: disaster_alert_model(
        row['update_spike_ratio'],
        row['risk_score']
    ),
    axis=1
)


In [123]:
merged['disaster_alert'].value_counts()


disaster_alert
NORMAL    4070
WATCH      328
Name: count, dtype: int64

# Aadhaar Kendra Optimization Policy Model

In [ ]:
def kendra_policy_model(p_risk, adult_stress, child_stress):
    
    if p_risk > 0.85:
        return "INCREASE_ADULT_CAPACITY" if adult_stress > child_stress else "CHILD_UPDATE_CAMPS"
    elif p_risk > 0.6:
        return "DEPLOY_MOBILE_UNIT"
    elif p_risk > 0.4:
        return "OFF_PEAK_ADVISORY"
    else:
        return "NORMAL_OPERATIONS"


In [125]:
merged['kendra_action'] = merged.apply(
    lambda row: kendra_policy_model(
        row['risk_score'],
        row['adult_system_stress'],
        row['child_system_stress']
    ),
    axis=1
)

merged['kendra_action'].value_counts()


kendra_action
NORMAL_OPERATIONS          4341
INCREASE_ADULT_CAPACITY      18
DEPLOY_MOBILE_UNIT           16
OFF_PEAK_ADVISORY            15
CHILD_UPDATE_CAMPS            8
Name: count, dtype: int64

In [126]:
final_outputs = merged[
    [
        'state',
        'district',
        'month',
        'risk_score',
        'age_verification_decision',
        'disaster_alert',
        'kendra_action'
    ]
]

final_outputs.to_csv("../Outputs/final_decision_outputs.csv", index=False)
final_outputs

,state,district,month,risk_score,age_verification_decision,disaster_alert,kendra_action
0,Andaman and Nicobar Islands,nicobar,9,0.007412,AGE_NOT_VERIFIED,NORMAL,NORMAL_OPERATIONS
1,Andaman and Nicobar Islands,nicobar,10,0.001601,AGE_NOT_VERIFIED,NORMAL,NORMAL_OPERATIONS
2,Andaman and Nicobar Islands,nicobar,11,0.025498,AGE_NOT_VERIFIED,NORMAL,NORMAL_OPERATIONS
3,Andaman and Nicobar Islands,nicobar,12,0.000987,AGE_NOT_VERIFIED,NORMAL,NORMAL_OPERATIONS
4,Andaman and Nicobar Islands,north and middle andaman,9,0.858008,DEFER_VERIFICATION,NORMAL,CHILD_UPDATE_CAMPS
...,...,...,...,...,...,...,...
4393,West Bengal,uttar dinajpur,12,NaN,AGE_VERIFIED_18_PLUS,NORMAL,NORMAL_OPERATIONS
4394,West Bengal,west midnapore,9,NaN,AGE_NOT_VERIFIED,NORMAL,NORMAL_OPERATIONS
4395,West Bengal,west midnapore,10,NaN,AGE_NOT_VERIFIED,NORMAL,NORMAL_OPERATIONS
4396,West Bengal,west midnapore,11,NaN,AGE_NOT_VERIFIED,NORMAL,NORMAL_OPERATIONS
